In [1]:
from typing import Annotated, Literal

from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_experimental.tools import PythonREPLTool
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_openai import ChatOpenAI
from pydantic import BaseModel
from langchain_core.messages import HumanMessage
import os
import keyring

# API KEY
OPENAI_API_KEY = keyring.get_password('openai', 'key_for_mac')
TAVILY_API_KEY = keyring.get_password('tavily', 'key_for_mac')
os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY
os.environ['TAVILY_API_KEY'] = TAVILY_API_KEY

# Tavily tool
tavily_tool = TavilySearchResults(max_results=5)

# Code execution tool
python_repl_tool = PythonREPLTool()

# Set up LangSmith observability
os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGCHAIN_ENDPOINT'] = "https://api.smith.langchain.com"
os.environ['LANGCHAIN_API_KEY'] = keyring.get_password('langsmith', 'learning_agent')
os.environ['LANGCHAIN_PROJECT'] = "pr-stupendous-hood-8"


In [2]:
def agent_node(state, agent, name):
    result = agent.invoke(state)
    return {
        "messages": [HumanMessage(content=result["messages"][-1].content, name=name)]
    }
    
members = ["Researcher", "Coder"]
system_prompt = (
    "You are a supervisor tasked with managing a conversation between the"
    " following workers: {members}. Given the following user request,"
    " respond with the worker to act next. Each worker will perfrom a"
    " task and respond with their results and status. When finished,"
    " respond with FINISH."
)

# Out team supervisor is an LLM node. It just picks the next agent to process
# and decides when the work is completed
options = ["FINISH"] + members


In [3]:
class routeResponse(BaseModel):
    next: Literal[tuple(options)]

In [4]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder(variable_name="messages"),
        (
            "system",
            "Given the conversation above, who should be next?"
            " or should we FINISH? Select one of: {options}",
        ),
    ]
).partial(options=str(options), members=", ".join(members))

llm = ChatOpenAI(model='gpt-4o-mini')

In [5]:
def supervisor_agent(state):
    supervisor_chain = prompt | llm.with_structured_output(routeResponse)
    return supervisor_chain.invoke(state)

In [6]:
import functools
import operator
from typing import Sequence, TypedDict
from langchain_core.messages import BaseMessage
from display_graph import display_graph

from langgraph.graph import StateGraph, START, END
from langgraph.prebuilt import create_react_agent

# The agent state is in the input to each node in the graph
class AgentState(TypedDict):
    # The annotation tells the graph that new messages will always
    # be added to the current states
    messages: Annotated[Sequence[BaseMessage], operator.add]
    # The 'next' field indicates where to route to next
    next: str
    
research_agent = create_react_agent(llm, tools=[tavily_tool])
research_node = functools.partial(agent_node, agent=research_agent, name="Reseacher")

# NOTE: THIS PERFORMS ARBITRARY CODE EXECUTION. PROCEED WITH CAUTION
code_agent = create_react_agent(llm, tools=[python_repl_tool])
code_node = functools.partial(agent_node, agent=code_agent, name="Coder")

In [7]:
# Build the graph
workflow = StateGraph(AgentState)
workflow.add_node("Researcher", research_node)
workflow.add_node("Coder", code_node)
workflow.add_node("Supervisor", supervisor_agent)

for member in members:
    # We want out worker to ALWAYS "report back" to the supervisor when done
    workflow.add_edge(member, "Supervisor")

# The supervisor populates the "next" fields in the graph state
# which routes to a node or finishes
conditional_map = {k: k for k in members}
conditional_map["FINISH"] = END
workflow.add_conditional_edges("Supervisor", lambda x: x["next"], conditional_map)

# Finally add entrypoint
workflow.add_edge(START, "Supervisor")

graph = workflow.compile()

# Visualize the graph
display_graph(graph)

/Users/woojin/Documents/github/learning/llm/langgraph/langgraph_blueprint/ch13/graphs/graph_56951.png


In [8]:
for s in graph.stream(
    {"messages": [
        HumanMessage(content="Research recent advances in AI and summarize.")
    ]}
):
    if "__end__" not in s:
        print(s)
        print("----")

{'Supervisor': {'next': 'Researcher'}}
----
{'Researcher': {'messages': [HumanMessage(content='Recent advances in AI during 2023 have been significant and varied, highlighting rapid developments across multiple domains. Here are some key takeaways:\n\n1. **Generative AI Models**: The rise of generative AI models has been a major focus, with practical applications becoming more integrated into everyday use. This includes enhanced natural language processing capabilities and improved interaction between users and AI systems like ChatGPT.\n\n2. **AI in Business and Services**: AI tools, such as "Copilots," are now being embedded into business software, assisting in complex tasks ranging from security management to search functions. This has made AI services more accessible to millions of users.\n\n3. **Autonomous Systems**: There have been notable advancements in autonomous systems, which are reshaping various industries. These advancements are critical for applications ranging from self-

In [9]:
for s in graph.stream(
    {"messages": [HumanMessage(content="Write a brief pdf researh report on pikas.")]},
    {"recursion_limit": 100},
):
    if "__end__" not in s:
        print(s)
        print("----")

{'Supervisor': {'next': 'Researcher'}}
----
{'Researcher': {'messages': [HumanMessage(content='# Research Report on Pikas\n\n## Introduction\n\nPikas (family Ochotonidae) are small mammals closely related to rabbits and hares. They inhabit mountainous regions in North America, Asia, and parts of Europe. Known for their distinctive, small size and round ears, pikas play a crucial ecological role in their environments, often serving as indicators of climate change due to their sensitivity to temperature.\n\n## Physical Characteristics\n\nPikas are typically 15-30 cm long and weigh between 120-500 grams. They have a stocky body covered with dense fur that can vary in color from brown to gray, allowing them to blend into their rocky habitats. Their short limbs and rounded ears help them adapt to cold climates. \n\n## Habitat\n\nPikas are predominantly found in alpine and subalpine regions, where they inhabit rocky slopes and talus fields. They require cool environments to thrive, as they a

Python REPL can execute arbitrary code. Use with caution.


{'Coder': {'messages': [HumanMessage(content='It seems that I am currently unable to create a PDF file due to missing libraries in the environment. However, I can guide you on how to create a PDF report using Python on your local machine.\n\nHere are the steps to create a PDF report on pikas using the `fpdf` library:\n\n1. **Install the `fpdf` library**:\n   Open your terminal or command prompt and run:\n   ```\n   pip install fpdf\n   ```\n\n2. **Use the following Python code** to generate the PDF:\n\n   ```python\n   from fpdf import FPDF\n\n   class PDF(FPDF):\n       def header(self):\n           self.set_font(\'Arial\', \'B\', 12)\n           self.cell(0, 10, \'Research Report on Pikas\', 0, 1, \'C\')\n\n       def footer(self):\n           self.set_y(-15)\n           self.set_font(\'Arial\', \'I\', 8)\n           self.cell(0, 10, f\'Page {self.page_no()}\', 0, 0, \'C\')\n\n   pdf = PDF()\n   pdf.add_page()\n   pdf.set_font(\'Arial\', \'\', 12)\n\n   content = [\n       \'## Intro